In [1]:
library(Seurat)
library(BisqueRNA)
library(fastSave)
library(pbmcapply)


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, were retired in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.

Attaching SeuratObject

Loading required package: parallel



In [2]:
# load("~/代表性方法测试/BayesPrism/tutorial/tutorial.gbm.rdata")
combine.dgedata_150=readRDS.lbzip2('~/ReCIDE/benchmark测试/high_res_TNBC/ref_data/ref_all.rdsFS',n.cores = 200)
patient_names=unique(combine.dgedata_150@meta.data['batch'])



In [4]:
bulk.mtx<-0
for (j in 1:nrow(patient_names)) {
  bulk_seurat<-subset(combine.dgedata_150,batch == patient_names[j,1])
  bulk.mtx1<-as.matrix(apply(as.data.frame(bulk_seurat@assays[["RNA"]]@counts),1,sum))
  if(length(bulk.mtx) != 1){
    bulk.mtx<-cbind(bulk.mtx,bulk.mtx1)}else{
      bulk.mtx<-bulk.mtx1}
}

bulk.mtx<-as.matrix(bulk.mtx)
colnames(bulk.mtx)<-patient_names[,1]
bulk.eset <- Biobase::ExpressionSet(assayData = bulk.mtx)

In [5]:

##############################################
function_bisque_in<-function(i){
  sc_seurat<-subset(combine.dgedata_150,batch != patient_names[i,1])
  # sc_seurat=sc_seurat[row.names(ref_com.FC_screen[[i]]),]
  cell_num=apply(sc_seurat@assays$RNA@counts,2,sum)
  cell_num=cell_num[cell_num>0]
  sc_seurat=sc_seurat[,names(cell_num)]
    gc()
  sample.ids <- colnames(sc_seurat)
  individual.labels<-sc_seurat@meta.data['batch']
  cell.type.labels<-sc_seurat@meta.data['celltype_subset']
  
  sc.pheno <- data.frame(check.names=F, check.rows=F,
                         stringsAsFactors=F,
                         row.names=sample.ids,
                         SubjectName=individual.labels,
                         cellType=cell.type.labels)
  
  sc.meta <- data.frame(labelDescription=c("SubjectName",
                                           "cellType"),
                        row.names=c("SubjectName",
                                    "cellType"))
  sc.pdata <- new("AnnotatedDataFrame",
                  data=sc.pheno,
                  varMetadata=sc.meta)
  
  sc.eset <- Biobase::ExpressionSet(assayData=as.matrix(sc_seurat@assays$RNA@counts),
                                    phenoData=sc.pdata)
    gc()
    
  colnames(sc.eset@phenoData@data)<-c("SubjectName","cellType")
    
    
  res <- BisqueRNA::ReferenceBasedDecomposition(bulk.eset, sc.eset, markers=NULL, use.overlap=FALSE)
  results<-as.data.frame(res[["bulk.props"]][,as.character(patient_names[,1])])
  colnames(results)<-patient_names[i,1]
  return(results)
}
results_bisque<-pbmclapply(1:nrow(patient_names),function_bisque_in,mc.cores = 5)

names(results_bisque)<-patient_names[,1]





In [6]:
results_bisque[[1]]

,CID3946
,<dbl>
B-cells,0.000000000
CAFs,0.121930352
Cancer Epithelial,0.163006644
Endothelial,0.110276193
Myeloid,0.139229064
Normal Epithelial,0.002632336
PVL,0.198820136
Plasmablasts,0.012043660
T-cells,0.252061617


In [7]:
saveRDS(results_bisque,file = '~/ReCIDE/benchmark测试/high_res_TNBC/bisque/bisque_tutorial/bisuqe_output.rds')


In [43]:

prd<-results_bisque

df_merge<-as.data.frame(prd[[1]])
# colnames(df_merge)[1]<-names(prd)[1]
colnames(df_merge)[1]<-names(prd[[1]])

for(j in 2:length(prd)){
  
  prd[[j]]<-as.data.frame(prd[[j]])
  df_merge<-merge(df_merge, prd[[j]], by = "row.names", all = TRUE)
  row.names(df_merge)<-df_merge[,1]
  df_merge<-df_merge[,-1]
  # colnames(df_merge)[j]<-names(prd)[j]
  colnames(df_merge)[j]<-names(prd[[j]])
}



df_merge<-as.data.frame(df_merge)
df_merge<-df_merge[,sort(colnames(df_merge))]
df_merge<-df_merge[sort(row.names(df_merge)),]


saveRDS(df_merge,file='~/ReCIDE/benchmark测试/high_res_TNBC/bisque/bisque_tutorial/prd_bisque.rds')


In [9]:
prd_df

,1153EO1,1153EO2,1153EO3,1157EO1,1157EO2,1157EO3,1158EO1,1158EO2,1158EO3,1162EO1,⋯,34-10050,34-10184,34-10187,PRE018,PRE019,PRE027,PRE038,PRE055,PRE062,PRE098
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ATL,0.000000000,0.000000000,0.000000000,0.001881823,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.01294012,0.038555786,0.000000000,0.000000000,0.000000000,0.004511949,0.03684482,0.042305841,0.000000000,0.00000000
CNT,0.152813681,0.178218216,0.092507096,0.028071210,0.044223551,0.026095120,0.010532554,0.036583514,0.071465157,0.082309635,⋯,0.02287242,0.002996620,0.019503612,0.153819164,0.104603965,0.090411800,0.03531406,0.021080072,0.150707356,0.22145961
DCT,0.095568349,0.110408022,0.055126826,0.063984313,0.047442429,0.014988231,0.000000000,0.016763389,0.099489058,0.048042939,⋯,0.01681183,0.005241242,0.007927715,0.089300415,0.136917918,0.195655682,0.02026266,0.038354514,0.141059583,0.08883898
DTL,0.000000000,0.000000000,0.000000000,0.009212033,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.02243024,0.123894661,0.000000000,0.037080975,0.000000000,0.012585439,0.06004156,0.000000000,0.000000000,0.00000000
EC,0.165007546,0.089372652,0.132530689,0.096871408,0.083080599,0.074824703,0.121537096,0.069262038,0.146908778,0.121269628,⋯,0.05023950,0.051347822,0.069254494,0.091642104,0.130059614,0.138166408,0.15280609,0.161055483,0.061305692,0.07981827
FIB,0.002517891,0.000000000,0.001948061,0.008961759,0.012736488,0.003574620,0.000000000,0.007246843,0.000000000,0.007191423,⋯,0.00000000,0.001843586,0.002602011,0.013267707,0.003462844,0.003619215,0.02740100,0.002149381,0.012484237,0.00890942
IC,0.075635084,0.080196823,0.078194133,0.262694025,0.038159468,0.025141842,0.008590524,0.030203901,0.097324927,0.077311467,⋯,0.08138942,0.050171059,0.023903643,0.049058750,0.077266129,0.131578109,0.04913189,0.065843768,0.089025003,0.08826907
IMM,0.114120392,0.084685776,0.121311519,0.089050956,0.090982311,0.081251447,0.053229996,0.087337077,0.055531604,0.058846507,⋯,0.36745236,0.084533926,0.087412113,0.080886671,0.065289864,0.062900215,0.10724315,0.165058658,0.116835516,0.07572782
PC,0.069980106,0.050209510,0.050176622,0.009964254,0.047948341,0.075157920,0.066514272,0.055338744,0.067568024,0.059296378,⋯,0.01438633,0.204497362,0.083593586,0.076129412,0.045029769,0.019654740,0.01956493,0.033008936,0.019336054,0.02270200


In [10]:
key_df

,1153EO1,1153EO2,1153EO3,1157EO1,1157EO2,1157EO3,1158EO1,1158EO2,1158EO3,1162EO1,⋯,34-10050,34-10184,34-10187,PRE018,PRE019,PRE027,PRE038,PRE055,PRE062,PRE098
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ATL,0.000000000,0.00000000,0.000000000,0.0000000000,0.0000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.0000000000,0.0026485055,0.000000000,0.000000000,0.0000000000,0.0000000000,0.0000000000,0.000000000,0.00000000,0.0000000000
CNT,0.208430913,0.27400468,0.077662722,0.0664767331,0.0168350168,0.014666667,0.007692308,0.063851699,0.109743590,0.063072228,⋯,0.0719310188,0.0022701476,0.019286967,0.074889868,0.1120331950,0.1197802198,0.0034782609,0.049572650,0.20502646,0.2148409894
DCT,0.101873536,0.12529274,0.064349112,0.0968660969,0.0361952862,0.025333333,0.000000000,0.009268795,0.103589744,0.077314344,⋯,0.0106648514,0.0003783579,0.010812390,0.097797357,0.1203319502,0.1659340659,0.0052173913,0.011965812,0.10978836,0.0833922261
DTL,0.000000000,0.00000000,0.000000000,0.0000000000,0.0016835017,0.000000000,0.015384615,0.001029866,0.000000000,0.000000000,⋯,0.0002269117,0.3129020053,0.009059030,0.000000000,0.0000000000,0.0000000000,0.0000000000,0.001709402,0.00000000,0.0000000000
EC,0.172131148,0.07962529,0.167159763,0.0959164292,0.0892255892,0.093333333,0.198076923,0.038105046,0.191794872,0.149542218,⋯,0.0501474926,0.0291335603,0.094097019,0.117180617,0.1514522822,0.1076923077,0.1495652174,0.182905983,0.07804233,0.0911660777
FIB,0.000000000,0.00000000,0.009615385,0.0009496676,0.0008417508,0.001333333,0.000000000,0.002059732,0.001025641,0.001017294,⋯,0.0052189698,0.0000000000,0.002337814,0.022026432,0.0011855365,0.0005494505,0.0005797101,0.005128205,0.03571429,0.0007067138
IC,0.099531616,0.16861827,0.105029586,0.2412155745,0.0269360269,0.070666667,0.011538462,0.015447992,0.122051282,0.104781282,⋯,0.0238257318,0.0155126750,0.015780245,0.079295154,0.1081802015,0.2000000000,0.0098550725,0.037606838,0.09126984,0.1455830389
IMM,0.125292740,0.07377049,0.180473373,0.0721747388,0.0336700337,0.093333333,0.028846154,0.035015448,0.032820513,0.017293998,⋯,0.3748581802,0.0298902762,0.077440094,0.084581498,0.0646117368,0.0241758242,0.0463768116,0.203418803,0.15079365,0.0416961131
NEU,0.000000000,0.00000000,0.000000000,0.0000000000,0.0000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.0000000000,0.0000000000,0.000000000,0.000000000,0.0002963841,0.0000000000,0.0000000000,0.000000000,0.00000000,0.0000000000


In [44]:
prd_df=df_merge
key_df <- readRDS("~/ReCIDE/benchmark测试/high_res_TNBC/EXP_and_KEY/KEY.rds")


In [31]:
# prd_df['NEU',]=0

In [45]:
prd_df=prd_df[sort(row.names(prd_df)),sort(colnames(prd_df))]
key_df=key_df[sort(row.names(key_df)),sort(colnames(prd_df))]

In [46]:

RMSE_vec=c()
for(i in 1:ncol(key_df)){
  RMSE_vec[i]=ModelMetrics::rmse(as.numeric(key_df[,i]),as.numeric(prd_df[,i]))
  # RMSE_vec[i]=cor(key_df[,i],prd_df[,i])
  
}
median(RMSE_vec)

[1] 0.0298466

In [47]:
key_df

,ROSMAP-10859,ROSMAP-12078,ROSMAP-14589,ROSMAP-15050,ROSMAP-15253,ROSMAP-16518,ROSMAP-17199,ROSMAP-17378,ROSMAP-17559,ROSMAP-18695,⋯,ROSMAP-83352,ROSMAP-83589,ROSMAP-84918,ROSMAP-85806,ROSMAP-86126,ROSMAP-87836,ROSMAP-96129,ROSMAP-97522,ROSMAP-98683,ROSMAP-99419
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ast,0.089525515,0.048471616,0.060007318,0.031945441,0.043850973,0.121171171,0.121141719,0.114048730,0.086117137,0.106393245,⋯,0.029320139,0.060514372,0.063857374,0.042049568,0.072498662,0.111510791,0.049087221,0.129162331,0.040686586,0.085935548
Exc,0.279021188,0.183842795,0.244968899,0.113244795,0.118694362,0.196171171,0.515433123,0.250388802,0.275054230,0.289022919,⋯,0.359354957,0.368465288,0.537763371,0.380952381,0.525147138,0.411586520,0.567951318,0.281477627,0.345835982,0.515113665
Inh,0.150104446,0.179475983,0.095682400,0.065685571,0.109792285,0.096396396,0.119482244,0.126836012,0.180477223,0.216405308,⋯,0.236576874,0.201714574,0.230794165,0.140907825,0.137774211,0.115107914,0.206490872,0.042793965,0.243165925,0.121908569
Mic,0.039391226,0.060043668,0.050311014,0.034278536,0.041872733,0.036936937,0.015267176,0.034560221,0.053145336,0.035946924,⋯,0.036650174,0.044041015,0.006482982,0.005847953,0.003477796,0.014577811,0.004056795,0.107830385,0.064844247,0.006495129
Oli,0.373619815,0.461572052,0.504390779,0.716259871,0.649521925,0.494594595,0.146033853,0.435804389,0.332537961,0.294330519,⋯,0.287520616,0.273827534,0.094651540,0.376496798,0.202514714,0.312381674,0.141176471,0.332075963,0.245390973,0.216087934
Opc,0.058788421,0.059825328,0.036406879,0.036611630,0.032640950,0.048648649,0.074344507,0.034560221,0.067895879,0.051145959,⋯,0.046179219,0.047234829,0.064505673,0.047619048,0.055912253,0.027546384,0.030020284,0.099505723,0.054036872,0.052460655
Vas,0.009549388,0.006768559,0.008232711,0.001974156,0.003626772,0.006081081,0.008297378,0.003801624,0.004772234,0.006755127,⋯,0.004398021,0.004202387,0.001944895,0.006126427,0.002675227,0.007288906,0.001217039,0.007154006,0.006039415,0.001998501


In [28]:
prd_df

,1153EO1,1153EO2,1153EO3,1157EO1,1157EO2,1157EO3,1158EO1,1158EO2,1158EO3,1162EO1,⋯,34-10050,34-10184,34-10187,PRE018,PRE019,PRE027,PRE038,PRE055,PRE062,PRE098
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ATL,0.000000000,0.000000000,0.000000000,0.001881823,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.01294012,0.038555786,0.000000000,0.000000000,0.000000000,0.004511949,0.03684482,0.042305841,0.000000000,0.00000000
CNT,0.152813681,0.178218216,0.092507096,0.028071210,0.044223551,0.026095120,0.010532554,0.036583514,0.071465157,0.082309635,⋯,0.02287242,0.002996620,0.019503612,0.153819164,0.104603965,0.090411800,0.03531406,0.021080072,0.150707356,0.22145961
DCT,0.095568349,0.110408022,0.055126826,0.063984313,0.047442429,0.014988231,0.000000000,0.016763389,0.099489058,0.048042939,⋯,0.01681183,0.005241242,0.007927715,0.089300415,0.136917918,0.195655682,0.02026266,0.038354514,0.141059583,0.08883898
DTL,0.000000000,0.000000000,0.000000000,0.009212033,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.02243024,0.123894661,0.000000000,0.037080975,0.000000000,0.012585439,0.06004156,0.000000000,0.000000000,0.00000000
EC,0.165007546,0.089372652,0.132530689,0.096871408,0.083080599,0.074824703,0.121537096,0.069262038,0.146908778,0.121269628,⋯,0.05023950,0.051347822,0.069254494,0.091642104,0.130059614,0.138166408,0.15280609,0.161055483,0.061305692,0.07981827
FIB,0.002517891,0.000000000,0.001948061,0.008961759,0.012736488,0.003574620,0.000000000,0.007246843,0.000000000,0.007191423,⋯,0.00000000,0.001843586,0.002602011,0.013267707,0.003462844,0.003619215,0.02740100,0.002149381,0.012484237,0.00890942
IC,0.075635084,0.080196823,0.078194133,0.262694025,0.038159468,0.025141842,0.008590524,0.030203901,0.097324927,0.077311467,⋯,0.08138942,0.050171059,0.023903643,0.049058750,0.077266129,0.131578109,0.04913189,0.065843768,0.089025003,0.08826907
IMM,0.114120392,0.084685776,0.121311519,0.089050956,0.090982311,0.081251447,0.053229996,0.087337077,0.055531604,0.058846507,⋯,0.36745236,0.084533926,0.087412113,0.080886671,0.065289864,0.062900215,0.10724315,0.165058658,0.116835516,0.07572782
PC,0.069980106,0.050209510,0.050176622,0.009964254,0.047948341,0.075157920,0.066514272,0.055338744,0.067568024,0.059296378,⋯,0.01438633,0.204497362,0.083593586,0.076129412,0.045029769,0.019654740,0.01956493,0.033008936,0.019336054,0.02270200


In [25]:
RMSE_vec

[1] 0.06614162 0.06939511 0.09674809 0.08794976 0.16267524 0.17068838
 [7] 0.17879419 0.18283505 0.09066519 0.13187724 0.11209034 0.14410569
[13] 0.11772486 0.09921674 0.15748049 0.10694880 0.08307967 0.08840949
[19] 0.13750885 0.11872311 0.13926095 0.10712316 0.11866564 0.10088756
[25] 0.11229258 0.15342575 0.15294194 0.10819350 0.13356977 0.08477810
[31] 0.08401542 0.10251550 0.20870825 0.10463703 0.14203574 0.17325853
[37] 0.09588449 0.08113188 0.08975136 0.17069614        NaN 0.06897503
[43] 0.08021633

In [5]:
key_df <- readRDS("~/ReCIDE/benchmark测试/high_res_TNBC/EXP_and_KEY/KEY.rds")
prd_df <- readRDS("~/ReCIDE/benchmark测试/high_res_TNBC/bisque/bisque_tutorial/prd_bisque.rds")
prd_df[is.na(prd_df)]=0

prd_df=prd_df[sort(row.names(prd_df)),sort(colnames(prd_df))]
key_df=key_df[sort(row.names(key_df)),sort(colnames(prd_df))]

RMSE_vec=c()
for(i in 1:ncol(key_df)){
  RMSE_vec[i]=ModelMetrics::rmse(as.numeric(key_df[,i]),as.numeric(prd_df[,i]))
  # RMSE_vec[i]=cor(key_df[,i],prd_df[,i])
  
}
mean(RMSE_vec)

[1] 0.04433162

[1] 2

In [2]:
eval(1+1)

[1] 2